# Install packeges 

``pip install requests``

``pip install python-certifi-win32``

# REST API with ``requests``

### HTTP-Methoden 
- GET
- POST
- PUT
- DELETE

### Statuscodes
#### 2xx - Succes 
- 200 : OK
- 201 : Created
#### 4xx - Client error
- 400 : Bad Request
- 401 : Unauthorized
- 403 : Forbidden
- 404 : Not Found
#### 5xx - Server error
- 500 : Internal Server Error
- 502 : Bad Gateway


## quick example

### Ping

In [9]:
import requests

# Disable SSL warnings
import urllib3
urllib3.disable_warnings()

urls = {
    'BASF': 'https://www.basf.com',                                                   # Our BASF homepage
    'LDAP': 'https://app.roqs.basf.net/ldap/api/v2'                                   # A BASF LDAP API
}
response_text = {}
for nickname, url in urls.items():
    print (nickname, end = '')
    response = requests.get(url,verify=False)
    print ('', response.status_code, response.reason, response.elapsed.total_seconds())
    #print(response.text)
    if response.ok:
        response_text[nickname] = response.text

BASF 200 OK 0.047835
LDAP 200 OK 0.088234


### get something practical

In [10]:
import os
import pandas as pd

userid = os.environ['USERNAME']
print('windows userid:', userid)

base_url = urls['LDAP']
end_point = 'users'
query = 'username=' + userid

response = requests.get(f'{base_url}/{end_point}?{query}', verify=False)
if not response.ok:
    print ("Respose Code:",response.status_code,"\n Response Reason:", response.reason)
    print ('request took %.3f seconds' % response.elapsed.total_seconds())
display(response.json())
pd.DataFrame(response.json())

windows userid: AlGhanH


[{'dn': 'cn=AlGhanH,ou=BASF IT SERVICES GMBH,ou=EMPLOYEES,o=AUTH',
  'building': 'Lu-Pfalzgrafenstrasse 1',
  'company': 'BASF Digital Solutions GmbH',
  'company_id': '0074',
  'cost_center': '3R613101',
  'country': 'Germany',
  'country_code': 'DE',
  'first_name': 'Hashem',
  'full_name': 'Hashem Al Ghanem',
  'gender': 'male',
  'id': '1002115746',
  'last_name': 'Al Ghanem',
  'login_disabled': 'FALSE',
  'mail': 'hashem.al-ghanem@basf.com',
  'org_code': 'GD/HP',
  'person_id': '01006843',
  'postal_code': '67061',
  'region': 'EU',
  'room': '032W20',
  'site': 'Ludwigshafen am Rhein',
  'state': 'Rheinland-Pfalz',
  'street': 'Pfalzgrafenstrasse 1',
  'supervisor_id': '1000513296',
  'username': 'AlGhanH',
  'type': 'employee',
  'status': 'active',
  'image': 'https://app.roqs.basf.net/user_picture_api/image/AlGhanH'}]

,dn,building,company,company_id,cost_center,country,country_code,first_name,full_name,gender,...,region,room,site,state,street,supervisor_id,username,type,status,image
0,"cn=AlGhanH,ou=BASF IT SERVICES GMBH,ou=EMPLOYE...",Lu-Pfalzgrafenstrasse 1,BASF Digital Solutions GmbH,0074,3R613101,Germany,DE,Hashem,Hashem Al Ghanem,male,...,EU,032W20,Ludwigshafen am Rhein,Rheinland-Pfalz,Pfalzgrafenstrasse 1,1000513296,AlGhanH,employee,active,https://app.roqs.basf.net/user_picture_api/ima...


## Anime Example

- [Jikan API](https://docs.api.jikan.moe/#tag/anime/operation/getAnimeSearch)

getAnimeSearch

| Query parameters 	| type    	| Description                                                                                              	|
|------------------	|---------	|----------------------------------------------------------------------------------------------------------	|
| page             	| integer 	| Page number.                                                                                          	|
| limit            	| integer 	| limit of the animes in the response.                                                                     	|
| type             	| string  	| Type of the animes in the response.<br>"tv" "movie" "ova" "special" "ona" "music" "cm" "pv" "tv_special" 	|
| rating           	| string  	| age rating.<br>"g" "pg" "pg13" "r17" "r" "rx"                                                            	|
| order_by         	| string  	| Order of the animes in the response.<br>"desc" "asc"                                                     	|


### Get data

In [11]:
import json
import requests

def get_anime_list(page=None, limit=None, type=None, rating=None, order_by=None):
    base_url = 'api.jikan.moe/v4/anime'
    query = '?'
    
    if page:
        query += f'&page={page}'
    if limit:
        query += f'&limit={limit}'
    if type:
        query += f'&type={type}'
    if rating:
        query += f'&rating={rating}'
    if order_by:
        query += f'&order_by={order_by}'
    
    if query == '?':
        query = ''

    url = f'https://{base_url}{query}'
    print(url)
    
    response = requests.get(url, verify=False)
    
    if not response.ok:
        print ("Respose Code:",response.status_code,"\n Response Reason:", response.reason)
        print ('request took %.3f seconds' % response.elapsed.total_seconds())


    with open('anime.json', 'w') as file:
        json.dump(response.json(), file)
    
    return response.json()

# get_anime_list()
animes_json = get_anime_list(limit=5)

https://api.jikan.moe/v4/anime?&limit=5


### work with data

In [14]:
import markdown

anime_count = animes_json['pagination']['items']['count']

anime_markdown_list = []

anime_list = animes_json['data']
for anime_item in anime_list:
    # Get the title
    anime_title = anime_item['title']
    
    # Get the image
    anime_image = anime_item['images']['jpg']['image_url']
    
    # Get the episode count
    anime_episode_count = anime_item['episodes']
    
    # Get the genres
    anime_genres = anime_item['genres']
    anime_genre_list=[]
    for genre in anime_genres:
        anime_genre_list.append(genre['name'])

    # Create the markdown string and append to the list
    anime_string = f"## {anime_title}<br>\n ![{anime_title}'image']({anime_image})<br>\n Episodes: {anime_episode_count}<br>\n Genres: {anime_genre_list}<br><br>\n"
    anime_markdown_list.append(anime_string)

# Create the final markdown string
animes_markdown = markdown.markdown(f'# My Anime list\n Count: {anime_count}<br>\n'+''.join(anime_markdown_list))

# Write to file
with open('myAnimeList.md', 'w') as file:
    file.write(animes_markdown)

## Dog or Cat

### Dog

create a markdown file with 10 random images of dogs using qurey parameters.
[find more here.](https://documenter.getpostman.com/view/4016432/the-dog-api/RW81vZ4Z#67a5dbff-cf14-4e60-98f6-d8a93d324fba)

API url:
https://api.thedogapi.com/v1/images/search

In [ ]:
# Code here


### Cat

create a markdown file with 10 random images of cats using qurey parameters.
[Find more here.](https://developers.thecatapi.com/view-account/ylX4blBYT9FaoVd6OhvR?report=bOoHBz-8t)

API url:
https://api.thecatapi.com/v1/images/search

In [ ]:
# Code here